# How to accept input for scripts from the command line

# Prep

In [1]:
! ls -al

total 76
drwxrwxr-x 5 rajiv rajiv  4096 Feb 10 05:47 .
drwxrwxr-x 5 rajiv rajiv  4096 Feb 10 05:17 ..
-rw-rw-r-- 1 rajiv rajiv   351 Feb  7 05:30 activity.py
-rw-rw-r-- 1 rajiv rajiv  1461 Feb  7 05:30 argparse_example.py
-rw-rw-r-- 1 rajiv rajiv    13 Feb  7 05:30 .gitignore
drwxrwxr-x 2 rajiv rajiv  4096 Feb  7 05:31 .ipynb_checkpoints
-rw-rw-r-- 1 rajiv rajiv 20166 Feb 10 05:47 presentation.ipynb
drwxrwxr-x 2 rajiv rajiv  4096 Feb 10 05:45 __pycache__
-rw-rw-r-- 1 rajiv rajiv     7 Feb  7 05:30 .python-version
-rw-rw-r-- 1 rajiv rajiv   211 Feb  7 05:30 README.md
-rw-rw-r-- 1 rajiv rajiv    34 Feb  7 05:30 requirements.txt
-rw-rw-r-- 1 rajiv rajiv    12 Feb  7 05:30 runtime.txt
-rw-rw-r-- 1 rajiv rajiv    73 Feb  7 05:30 simple.py
-rw-rw-r-- 1 rajiv rajiv   830 Feb  7 05:30 tasks.py
drwxrwxr-x 7 rajiv rajiv  4096 Feb  7 05:31 venv


## Beginner: Using inbuilt Python args

# simple.py
```python
import sys
if __name__ == '__main__':
    args = sys.argv
    print(args)
```

In [2]:
! python simple.py --activity=train 10 True /some/folder

['simple.py', '--activity=train', '10', 'True', '/some/folder']


# Opinion
* strictly positional. I can't change the order
* hard to know what `10 True..` stand for when one sees them in code. **Out of Band Documentation** check. 
* no interactive help commands to list tasks or add documentation like `ls --help`
* We can give things like `python simple.py --activity=train` but then I have do write code to extract that. E.g. `"--activity=train".split('=')`
* needless toil


## Using argparse

In [4]:
! python argparse_example.py -h

usage: argparse_example.py [-h] -a {train,predict} [-e EPOCHS]
                           [-d | --debug | --no-debug]
                           [-p | --profile | --no-profile]
                           [-l [LEARNING_RATE ...]] [-k PATH]

Train Model

options:
  -h, --help            show this help message and exit
  -a {train,predict}, --activity {train,predict}
                        train or predict
  -e EPOCHS, --epochs EPOCHS
                        number of epochs
  -d, --debug, --no-debug
                        print debug statements (default: False)
  -p, --profile, --no-profile
                        profile memory statements (default: True)
  -l [LEARNING_RATE ...], --learning_rate [LEARNING_RATE ...]
                        model learning rate
  -k PATH, --path PATH  output folder


# argparse.py
```python
import argparse
from pathlib import Path
from activity import train

if __name__ == '__main__':
    parser = argparse.ArgumentParser(description="Train Model")
    
    ### Explained Below - Start
    # TODO: choice
    parser.add_argument("-a", "--activity", required=True, choices=['train', 'predict'], help="train or predict")
    # TODO: default and type
    parser.add_argument("-e", "--epochs", type=int, default=10, help="number of epochs")
    # TODO: Boolean options 1
    parser.add_argument('-d', '--debug', action=argparse.BooleanOptionalAction, default=False, help="print debug "
                                                                                                    "statements")  # Python 3.9+
    # TODO: Boolean options 2
    parser.add_argument('-p', '--profile', action=argparse.BooleanOptionalAction, default=True, help="profile memory"
                                                                                                     "statements")  # Python 3.9+
    # TODO: Multiple Values
    parser.add_argument('-l', '--learning_rate', nargs="*", help="model learning rate")

    # TODO: Can't give -p here as it'll conflict with `-p` for profile
    # TODO: type Path
    parser.add_argument("-k", "--path", type=Path, help="output folder")
    
    ### Explained Below - End

    args = parser.parse_args()

    if args.activity == 'train':
        train(epochs=args.epochs, debug=args.debug, profile=args.profile, learning_rates=args.learning_rate,
                       path=args.path)


```

In [5]:
# Defaults, Type Casting and Booleans
# parser.add_argument("-e", "--epochs", type=int, default=10, help="number of epochs")
# parser.add_argument('-d', '--debug', action=argparse.BooleanOptionalAction, default=False, help="print debug ""statements")  # Python 3.9+
   
! python argparse_example.py --activity=train --debug -e 11

In Training
epochs:11
type of epochs:<class 'int'>
debug:True
profile:True
learning rate:None
path: None
Type of path: <class 'NoneType'>


In [6]:
# invalid epoch
! python argparse_example.py --activity=train --debug -e not_a_number

usage: argparse_example.py [-h] -a {train,predict} [-e EPOCHS]
                           [-d | --debug | --no-debug]
                           [-p | --profile | --no-profile]
                           [-l [LEARNING_RATE ...]] [-k PATH]
argparse_example.py: error: argument -e/--epochs: invalid int value: 'not_a_number'


In [7]:
# no-debug
! python argparse_example.py --activity=train --no-debug -e 10

In Training
epochs:10
type of epochs:<class 'int'>
debug:False
profile:True
learning rate:None
path: None
Type of path: <class 'NoneType'>


In [10]:
# learning_rate. Multiple Values
# parser.add_argument('-l', '--learning_rate', nargs="*", help="model learning rate")
! python argparse_example.py --activity train -l 1 2 

In Training
epochs:10
type of epochs:<class 'int'>
debug:False
profile:True
learning rate:['1', '2']
path: None
Type of path: <class 'NoneType'>


In [11]:
# invalid choice
# parser.add_argument("-a", "--activity", required=True, choices=['train', 'predict'], help="train or predict")

! python argparse_example.py --activity bad_activity 

usage: argparse_example.py [-h] -a {train,predict} [-e EPOCHS]
                           [-d | --debug | --no-debug]
                           [-p | --profile | --no-profile]
                           [-l [LEARNING_RATE ...]] [-k PATH]
argparse_example.py: error: argument -a/--activity: invalid choice: 'bad_activity' (choose from 'train', 'predict')


In [12]:
# path
#  TODO: Can't give -p here as it'll conflict with `-p` for profile
# TODO: type Path
# parser.add_argument("-k", "--path", type=Path, help="output folder")
! python argparse_example.py --activity train --path /path/to/something

In Training
epochs:10
type of epochs:<class 'int'>
debug:False
profile:True
learning rate:None
path: /path/to/something
Type of path: <class 'pathlib.PosixPath'>


# Opinion
* part of standard library
* friction: hard to remember syntax 
* More info:    https://docs.python.org/3/library/argparse.html#quick-links-for-add-argument


## Invoke

pip install invoke

https://docs.pyinvoke.org/en/stable/getting-started.html

In [13]:
! invoke -l

Available tasks:

  do       Train or Predict Activity
  say-hi



# tasks.py
```python
from invoke import task
from activity import train

@task
def say_hi(context, name):
    # function names are underscore, cli invocations are hyphen
    # context object
    context.run(f"echo 'Hi {name}'")

    
    
@task(iterable=['learning_rate'])
def do(context, activity, epochs=20, debug=False, profile=True, learning_rate=None, path=None):
    """
    Train or Predict Activity

    :param context: Invoke context object
    :param activity: train or predict
    :param epochs: number of epochs
    :param debug: print debug statements
    :param profile: profile memory
    :param learning_rate: model learning rate
    :param path: output folder
    """

    context.run("echo 'Starting Training'")
    if activity == 'train':
        train(epochs=epochs, debug=debug, profile=profile, learning_rates=learning_rate, path=path)



```

In [14]:
! inv -h do

Usage: inv[oke] [--core-opts] do [--options] [other tasks here ...]

Docstring:
  Train or Predict Activity

  :param context: Invoke context object
  :param activity: train or predict
  :param epochs: number of epochs
  :param debug: print debug statements
  :param profile: profile memory
  :param learning_rate: model learning rate
  :param path: output folder

Options:
  -a STRING, --activity=STRING
  -d, --debug
  -e INT, --epochs=INT
  -l, --learning-rate
  -p, --[no-]profile
  -t STRING, --path=STRING



In [15]:
# learning rate. invoke vs argparse
# ! python argparse_example.py --activity train -l 1 2 
! inv do --activity=train -l 1 -l 2

Starting Training
In Training
epochs:20
type of epochs:<class 'int'>
debug:False
profile:True
learning rate:['1', '2']
path: None
Type of path: <class 'NoneType'>


In [16]:
# Boolean options
! inv do -a=train --debug --no-profile

Starting Training
In Training
epochs:20
type of epochs:<class 'int'>
debug:True
profile:False
learning rate:[]
path: None
Type of path: <class 'NoneType'>


In [17]:
# Path. as str
! inv do --activity train --path /path/to/something

Starting Training
In Training
epochs:20
type of epochs:<class 'int'>
debug:False
profile:True
learning rate:[]
path: /path/to/something
Type of path: <class 'str'>


In [28]:
! inv -h say-hi

Usage: inv[oke] [--core-opts] say-hi [--options] [other tasks here ...]

Docstring:
  none

Options:
  -n STRING, --name=STRING



In [29]:
! inv say-hi --name='Code Plumber'

Hi Code Plumber


# Context Object
* Contains "global" data i.e. values loaded from configuration files or other configuration vectors, given via CLI flags
* primary API endpoint to IO e.g. `context.run`
* Very powerful design pattern in general. It is seen in AWS Lambda and powerful languages like Pony(https://tutorial.ponylang.io/getting-started/hello-world.html)

## Limitations
        * specify type e.g. Path 
        * 'choices'.
`
## Strengths
    * Full fledged task runner. like make. tasks with dependencies.
    * Low friction

# Alternatives

- Click(https://click.palletsprojects.com/en/8.1.x/#). powerful but I like invoke's simplicity. Invoke is more of a task runner.
- pydoit(https://pydoit.org/). powerful but it has a style where I'm not sure I can convince my team.
- Google Fire(https://github.com/google/python-fire). Popular and like Click.
